# Cloud Optical Thickness Prediction from Imaging Satellite Instruments

In [1]:
import os
import math
import torch
from torch import nn
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import xarray as xr
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy.special import logit
from multiprocessing import Process
from time import perf_counter

import matplotlib.pyplot as plt

In [2]:
file_path ='data/rt_nn_cloud_training_data_20230518.nc'
ds = xr.open_dataset(file_path)
ds

<xarray.Dataset>
Dimensions:                        (npoints: 250000, nbands_oci: 223, nbands_modis: 11, nbands_viirs: 10)
Coordinates:
  * nbands_oci                     (nbands_oci) float32 350.0 ... 2.261e+03
  * nbands_modis                   (nbands_modis) float32 412.0 ... 2.13e+03
  * nbands_viirs                   (nbands_viirs) float32 412.0 ... 2.25e+03
Dimensions without coordinates: npoints
Data variables: (12/13)
    solar_zenith_angle             (npoints) float32 ...
    viewing_zenith_angle           (npoints) float32 ...
    relative_azimuth_angle         (npoints) float32 ...
    albedo_type                    (npoints) int16 ...
    oci_reflectances               (nbands_oci, npoints) float32 ...
    modis_reflectances             (nbands_modis, npoints) float32 ...
    ...                             ...
    scene_type                     (npoints) int16 ...
    cloud_type                     (npoints) int16 ...
    log10_cloud_optical_thickness  (npoints) float32 ...
    spress                         (npoints) float32 ...
    h2o                            (npoints) float32 ...
    o3                             (npoints) float32 ...
Attributes:
    description:             Radiative transfer-based labelled training data ...
    suffix:                  nc
    version:                 Created 20230518
    contact_person_address:  NASA/GSFC, Mail Code 616, 8800 Greenbelt Road, G...
    contact_person_email:     andrew.sayer@nasa.gov
    contact_person_name:      Andrew Sayer
    contact_person_role:     Scientific/technical contact
    data_set_language:       en
    file_format:             NetCDF4

In [3]:
# Define variables
oci_reflectances = ds["oci_reflectances"].values 
modis_reflectances = ds["modis_reflectances"].values 
viirs_reflectances = ds["viirs_reflectances"].values 
nbands_oci = ds["nbands_oci"].values
nbands_modis = ds["nbands_modis"].values
nbands_viirs = ds["nbands_viirs"].values
angles = ds[["solar_zenith_angle", "viewing_zenith_angle", "relative_azimuth_angle"]].to_array().values
h2o = ds["h2o"].values
o3 = ds["o3"].values
scene_type = ds["scene_type"].values
albedo_type = ds["albedo_type"].values
spress = ds["spress"].values
cot = ds["log10_cloud_optical_thickness"].values
cloud_type = ds["cloud_type"].values

### Data preprocess

In [7]:
# Define a minmax scaler
def minmaxscaler(reflectances):
        if len(reflectances.shape) == 1:
                reflectances_sc = (reflectances-reflectances.min())/(reflectances.max() - reflectances.min())
        else:
                reflectances_sc = []
                for i in range(len(reflectances)):
                        reflectances_sc_tem = (reflectances[i]-reflectances[i].min())/(reflectances[i].max()-reflectances[i].min())
                        reflectances_sc.append(reflectances_sc_tem)
        return np.array(reflectances_sc)
                

In [9]:
# Define labels for albedo type
land = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,18]
snow = [15,19,20]
desert = [16, 16]
ocean_water = [17, 17]
albedo = [land, snow, desert, ocean_water]

def albedo_category(albedo_type):
    albedo_type_sc = []
    for k in range(len(albedo)):
        i = 0
        albedo_1hot = []
        for i in range(len(albedo_type)):
            if albedo_type[i] in albedo[k]:
                albedo_1hot.append(1)
            else:
                albedo_1hot.append(0)
        albedo_type_sc.append(albedo_1hot)
    return np.array(albedo_type_sc)

In [10]:
"""
Reflectances, h2o, o3, spress: minmax normalize to [0,1]
Angles: take cosine and normalize to [0,1]
albedo_type: break down into 4 categories

y1 - scen_type: change values from [1, 2] to [0,1]
y2 - cot: set 0 as -2
"""
oci_reflectances_sc = minmaxscaler(oci_reflectances)
h2o_sc = minmaxscaler(h2o)
o3_sc = minmaxscaler(o3)
spress_sc = minmaxscaler(spress)
angles_sc = minmaxscaler(np.cos(angles))
albedo_type_sc = np.float32(albedo_category(albedo_type))

In [11]:
"""
scene_type: 1 cloud free; 2 single cloud layer
cloud_type: 1 liquid phase; 2 ice crystal
1-hot array: [0]: cloudy, [1]: liquid, [2]: ice
"""
scene_type_1hot = []
for i in range(len(scene_type)):
    if scene_type[i] == 1:
        scene_type_1hot.append([1,0,0])
    elif scene_type[i] == 2:
        if cloud_type[i] == 1:
            scene_type_1hot.append([0,1,0])
        elif cloud_type[i] == 2:
            scene_type_1hot.append([0,0,1])

In [16]:
transed_scene = np.transpose(scene_type_1hot)
# Count of cloud-free
print(f'Counts of cloud-free: {np.sum(transed_scene[0])} Percentage: {np.sum(transed_scene[0]) / len(scene_type)*100}')
# Count of cloud == 1
print(f'Counts of liquid phase: {np.sum(transed_scene[1])} Percentage: {np.sum(transed_scene[1]) / len(scene_type)*100}')
# Count of cloud == 2
np.sum(transed_scene[2])
print(f'Counts of ice crystal: {np.sum(transed_scene[2])} Percentage: {np.sum(transed_scene[2]) / len(scene_type)*100}')
# Count of cloud == 1 and cloud == 2
print(f'Sum of liquid and ice cloud: {np.sum(transed_scene[2])+ np.sum(transed_scene[1])}')

Counts of cloud-free: 75239 Percentage: 30.0956
Counts of liquid phase: 87362 Percentage: 34.9448
Counts of ice crystal: 87399 Percentage: 34.9596
Sum of liquid and ice cloud: 174761


In [17]:
scene_type_sc = minmaxscaler(np.array(scene_type_1hot))
scene_type_sc = np.float32(scene_type_sc)

cot_sc = np.nan_to_num(cot, nan=-2) 
max_value = cot_sc.max()
min_value = cot_sc.min()
cot_sc = minmaxscaler(cot_sc)

In [18]:
max_value

2.5

In [19]:
min_value

-2.0

## Concatenate variables and split training-testing dataset

In [20]:
X = np.concatenate((oci_reflectances_sc, h2o_sc[None, :], o3_sc[None, :], spress_sc[None, :], albedo_type_sc, angles_sc), axis=0) # n x m, where n = 22
# X = np.transpose(X) # transpose into a m x n matrix , n = 22 (aka features) and m = rows (aka observations)
X = np.float32(X)
# Y represents the response vector (binary),,,, m by 1, m = rows (aka observations)
Y_cls = scene_type_sc
Y_reg = cot_sc
Y = np.concatenate((Y_cls, np.transpose(Y_reg[None, :])), axis=1)

In [21]:
print(X.shape, Y.shape)

(233, 250000) (250000, 4)


In [22]:
X = np.transpose(X)

In [23]:
print(X.shape, Y.shape)

(250000, 233) (250000, 4)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) 

# Prepare Data for Training

In [25]:
X_train_t = torch.from_numpy(X_train.astype(np.float32))
y_train_t = torch.from_numpy(y_train.astype(np.float32))

X_val_t = torch.from_numpy(X_val.astype(np.float32))
y_val_t = torch.from_numpy(y_val.astype(np.float32))

X_test_t = torch.from_numpy(X_test.astype(np.float32))
y_test_t = torch.from_numpy(y_test.astype(np.float32))

In [26]:
torch.save(X_train_t, 'data/X_train_t.pt')
torch.save(y_train_t, 'data/y_train_t.pt')
torch.save(X_val_t, 'data/X_val_t.pt')
torch.save(y_val_t, 'data/y_val_t.pt')
torch.save(X_test_t, 'data/X_test_t.pt')
torch.save(y_test_t, 'data/y_test_t.pt')